In [1]:
import numpy as np
import os

from PIL import Image

import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import DataLoader as DataLoader
from torch.autograd import Variable



import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
IMAGE_H = 262
IMAGE_W = 262

In [3]:
data_transform = transforms.Compose([
    transforms.ToTensor()   # 转换成Tensor形式，并且数值归一化到[0.0, 1.0]
])

In [4]:
class DogsVSCatsDataset(data.Dataset):     
    def __init__(self, mode, dir):          
        self.mode = mode
        self.list_img = []                  
        self.list_label = []               
        self.data_size = 0                  
        self.transform = data_transform     

        if self.mode == 'train':            
            dir = dir + '/train/'           
            for file in os.listdir(dir):    
                self.list_img.append(dir + file)       
                self.data_size += 1                     
                name = file.split(sep='.')              
               
                if name[0] == 'cat':
                    self.list_label.append(0)        
                else:
                    self.list_label.append(1)  
                    
        elif self.mode == 'test':           
            dir = dir + '/test/'         
            for file in os.listdir(dir):
                self.list_img.append(dir + file)    
                self.data_size += 1
                self.list_label.append(2)   
                
        elif self.mode == 'valid':            
            dir = dir + '/valid/'           
            for file in os.listdir(dir):    
                self.list_img.append(dir + file)       
                self.data_size += 1                     
                name = file.split(sep='.')              
                
                if name[0] == 'cat':
                    self.list_label.append(0)         
                else:
                    self.list_label.append(1)         
        else:
            return print('Undefined Dataset!')

    def __getitem__(self, item):            
        if self.mode == 'train':                                        
            img = Image.open(self.list_img[item])                       
            img = img.resize((IMAGE_H, IMAGE_W))                        
            img = np.array(img)[:, :, :3]                               
            label = self.list_label[item]                               
            return self.transform(img), torch.LongTensor([label])       
        elif self.mode == 'test':                                      
            img = Image.open(self.list_img[item])
            img = img.resize((IMAGE_H, IMAGE_W))
            img = np.array(img)[:, :, :3]
            return self.transform(img)                                  
        if self.mode == 'valid':                                        
            img = Image.open(self.list_img[item])                       
            img = img.resize((IMAGE_H, IMAGE_W))                        
            img = np.array(img)[:, :, :3]                             
            label = self.list_label[item]                              
            return self.transform(img), torch.LongTensor([label])       
        else:
            print('None')

    def __len__(self):
        return self.data_size               # 返回数据集大小


In [5]:
DVCD = DogsVSCatsDataset

In [6]:
class Net(nn.Module):                                       
    def __init__(self):                                     
        super(Net, self).__init__()                         
        self.conv1 = torch.nn.Conv2d(3, 16, 3)   
        self.conv2 = torch.nn.Conv2d(16, 16, 3)  

        self.fc1 = nn.Linear(64*64*16, 64*8)                
        self.fc2 = nn.Linear(64*8, 64*2)                       
        self.fc3 = nn.Linear(64*2,64)                         
        self.fc4 = nn.Linear(64,2)  
    def forward(self, x):   
       
        x = self.conv1(x)                   
        x = F.relu(x)                      
        x = F.max_pool2d(x, 2)   
        
        x = self.conv2(x)                   
        x = F.relu(x)                       
        x = F.max_pool2d(x, 2)              
        
        x = x.view(x.size()[0], -1)        
        x = F.relu(self.fc1(x))             
        x = F.relu(self.fc2(x))             
        x = F.relu(self.fc3(x)) 
        x = self.fc4(x)

        return F.softmax(x, dim=1)  
       

In [7]:
dataset_dir = 'C:/ml_data/catsvsdogs/'             # 数据集路径
model_cp = 'C:/ml_data/catsvsdogs/'               # 网络参数保存位置
batch_size = 32                     # batch_size大小
lr = 0.0001  

In [ ]:
#这里是训练完，进行预测的代码

In [8]:
torch.cuda.empty_cache()

In [9]:
model_cnn_epoch_11 = torch.load('C:/ml_data/catsvsdogs/epoch_0.pth')

In [12]:
def pred(model):  
    with torch.no_grad():
        result = []
        datafile = DVCD('test', dataset_dir)  
        dataloader = DataLoader(datafile, batch_size=batch_size, shuffle=False)     
        print('Dataset loaded! length of test set is {0}'.format(len(datafile)))
        for img in dataloader:

            img= Variable(img).cuda()
            outputs = model(img).to('cpu')
            result += outputs
    return result
    

In [13]:
result = pred(model_cnn_epoch_11)

Dataset loaded! length of test set is 12500


In [23]:
predict_result = []
for item in result:
    predict_result.append(torch.argmax(item, dim=0).item())

In [ ]:
#预测结果： predict_result

In [18]:
#下面是训练的代码

In [8]:
model_cnn = Net().cuda()

torch.save(model_cnn,'C:/ml_data/catsvsdogs/model_init.pth')

In [12]:
model_cnn = torch.load('C:/ml_data/catsvsdogs/model_init.pth')

In [13]:
optimizer = torch.optim.Adam(model_cnn.parameters(), lr=lr) 
criterion = torch.nn.CrossEntropyLoss() 

In [14]:
def test(model, folder_name, criterion):                                      

    datafile = DVCD(folder_name, dataset_dir)  
    dataloader = DataLoader(datafile, batch_size=batch_size, shuffle=True)     
    print('Dataset loaded! length of test set is {0}'.format(len(datafile)))
    total = 0
    correct = 0
    total_loss = 0
    cnt = 0
    for img,label in dataloader:
        cnt += 1
        total += batch_size
        img= Variable(img).cuda()
        label = Variable(label).cuda() 
        
        outputs = model(img)
        loss = criterion(outputs, label.squeeze())
        total_loss += loss.item()
        
        pred_y = []
        for item in outputs:
            if item[0].item() > item[1].item():
                pred_y.append(0)
            else:
                pred_y.append(1)
        
        
        for i in range(len(label)):
            if pred_y[i] == label[i]:
                correct += 1
    return correct, total, total_loss/cnt
    
    


In [15]:
train_epoch_acc = []
train_epoch_loss = []
valid_epoch_acc = []
valid_epoch_loss = []

train_iter_loss = []
epoch_count = 0

In [17]:
def train(model, optimizer, criterion, epochs):
    global epoch_count
    epoch_count += 1
    datafile = DVCD('train', dataset_dir)                                                      
    dataloader = DataLoader(datafile, batch_size=batch_size, shuffle=True)     
    print('Dataset loaded! length of train set is {0}'.format(len(datafile)))
    
    for epoch in range(epochs):
        cnt = 0  
        for img, label in dataloader:                                          
            img, label = Variable(img).cuda(), Variable(label).cuda()           
            out = model(img) 
            
            loss = criterion(out, label.squeeze())      
            loss.backward()                             
            optimizer.step()                           
            optimizer.zero_grad()                      
            cnt += 1
            train_iter_loss.append(loss.item())
            if cnt % 100 == 0:
                print('Epoch: {0}/{1}, iter {2}/{3}, train_loss {4}'.format(epoch+1, epochs, cnt+1, int(20000/32)+1,loss/batch_size))   
            
        correct_count,total_count, valid_loss = test(model,'valid', criterion)
        valid_epoch_acc.append(correct_count/total_count*100)
        valid_epoch_loss.append(valid_loss)
        
        correct_count,total_count, train_loss = test(model,'train', criterion)
        train_epoch_acc.append(correct_count/total_count*100)
        train_epoch_loss.append(train_loss)
        torch.save(model,'C:/ml_data/catsvsdogs/epoch_{}.pth'.format(epoch_count))
    

In [ ]:
train(model_cnn, optimizer, criterion,11)

Dataset loaded! length of train set is 20000
Epoch: 1/11, iter 101/626, train_loss 0.021041983738541603
Epoch: 1/11, iter 201/626, train_loss 0.02064378932118416
Epoch: 1/11, iter 301/626, train_loss 0.020455822348594666
Epoch: 1/11, iter 401/626, train_loss 0.02094901166856289
Epoch: 1/11, iter 501/626, train_loss 0.021509183570742607
Epoch: 1/11, iter 601/626, train_loss 0.018877752125263214
Dataset loaded! length of test set is 5000
Dataset loaded! length of test set is 20000
Epoch: 2/11, iter 101/626, train_loss 0.021012594923377037
Epoch: 2/11, iter 201/626, train_loss 0.019397538155317307
Epoch: 2/11, iter 301/626, train_loss 0.02126147598028183
Epoch: 2/11, iter 401/626, train_loss 0.01920810341835022
Epoch: 2/11, iter 501/626, train_loss 0.020632024854421616
Epoch: 2/11, iter 601/626, train_loss 0.020515073090791702
Dataset loaded! length of test set is 5000
Dataset loaded! length of test set is 20000
Epoch: 3/11, iter 101/626, train_loss 0.017728760838508606
Epoch: 3/11, iter 

In [25]:
#epoch为11的时候，验证集acc最高,loss最小，所以设置epoch为11